In [41]:
import torch
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
import numpy as np
# np.argmax([1,2,3,4])

In [52]:
def softmax(x, axis=None):
    x = np.array(x)
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    rst = y / y.sum(axis=axis, keepdims=True)
    return rst.tolist()
softmax([1,2])

[0.2689414213699951, 0.7310585786300049]

In [18]:
tokenizer = AutoTokenizer.from_pretrained('ethanyt/guwenbert-base')

In [37]:
import json
with open('data/valid.jsonl','r') as f:
    valid_json = f.readlines()
    valid_json = [json.loads(e) for e in valid_json]

In [7]:
import pickle
with open('test_result.pkl','rb') as f:
    rst = pickle.load(f)

In [22]:
rst['test_dataset'],rst['test_dataset'][0].keys()

(Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'origin_idx', 'token_type_ids'],
     num_rows: 24797
 }),
 dict_keys(['attention_mask', 'input_ids', 'labels', 'origin_idx', 'token_type_ids']))

In [23]:
tokenizer.decode(rst['test_dataset'][0]['input_ids'])

'[CLS] 昏 暗 的 灯 熄 灭 了 又 被 重 新 点 亮 。 [SEP] 灭 又 明 [SEP]'

In [53]:
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = rst['test_dataset'][i]['origin_idx']
    input_str = rst['test_dataset'][i]['input_ids']
    input_str = tokenizer.decode(input_str)
    input_str = input_str.split('[SEP]')
    if idx not in all_scores:
        all_scores[idx] = {}
    all_scores[idx][input_str[1].replace(' ','')] = softmax(rst['test_result'].predictions[i].tolist())

100%|████████████████████████████████████████████████████████████████████████| 24797/24797 [00:10<00:00, 2392.46it/s]


In [38]:
valid_json[0]

{'translation': '昏暗的灯熄灭了又被重新点亮。',
 'choices': ['渔灯灭复明', '残灯灭又然', '残灯暗复明', '残灯灭又明'],
 'answer': 3}

3

In [62]:
predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = []
    for choice in choices:
        choice_score = 1
        for k in all_scores[i]:
            if k in choice:
                choice_score += all_scores[i][k][1]
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_dict['scores'] = this_scores
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

In [63]:
true_num = np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

In [77]:
predict_valid_json[-10]

{'translation': '请莫嫌弃潇湘一带人烟稀少。',
 'choices': ['莫厌潇湘少人处', '飞时莫近潇湘浦', '莫厌皖山穷绝处', '潇湘云水人何处'],
 'answer': 0,
 'scores': [2.951438719346828,
  2.0997304880167835,
  1.9610124685063726,
  2.1316768167366544],
 'predict': 0,
 'true': True}

In [64]:
true_num

0.8253676470588235

In [4]:
rst.predictions.shape

(24797, 2)

PredictionOutput(predictions=array([[ 0.03430561, -0.48208067],
       [ 0.09326166, -0.5414783 ],
       [ 0.33604988, -0.7945821 ],
       ...,
       [ 1.0819488 , -1.4716136 ],
       [ 1.0384777 , -1.4446024 ],
       [ 1.1046293 , -1.4878067 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 1]), metrics={'test_loss': 0.4463875889778137, 'test_accuracy': 0.8132838606834412, 'test_runtime': 28.3801, 'test_samples_per_second': 873.745, 'test_steps_per_second': 17.477})

In [80]:
sum(rst['test_result'].label_ids)/rst['test_result'].label_ids.shape[0]

0.30104448118724036

In [82]:
rst['test_result'].label_ids.shape

(24797,)

In [83]:
rst['test_result'].label_ids.sum()

7465